## keras seq2seq word-level

In [11]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding
from keras.utils.vis_utils import model_to_dot, plot_model

import numpy as np
import MeCab

In [2]:
batch_size = 64  # Batch size for training
epochs = 100  # Number of epochs to train for
latent_dim = 256  # Latent dimensionality of the encoding space.

input_data_path = 'data/aion_train_v.txt'
target_data_path = "data/aion_train_t.txt"

input_texts = []
target_texts = []
input_words = set()
target_words = set()

t = MeCab.Tagger('-d /usr/local/lib/mecab/dic/mecab-ko-dic')

with open(input_data_path, "r", encoding="utf-8") as f:
    reader = f.readlines()
    for sentence in reader:
        input_texts.append(sentence)
        words = t.parse(sentence).split()
        for word in words:
            if word.find(',') == -1 and word.find('EOS') == -1:
                if word not in input_words:
                    input_words.add(word)

with open(target_data_path, "r", encoding="utf-8") as f:
    reader = f.readlines()
    for sentence in reader:
        target_texts.append(sentence)
        words = t.parse(sentence).split()
        for word in words:
            if word.find(',') == -1 and word.find('EOS') == -1:
                if word not in target_words:
                    target_words.add(word)

In [3]:
input_words = sorted(list(input_words))
target_words = sorted(list(target_words))
num_encoder_tokens = len(input_words)
num_decoder_tokens = len(target_words)
max_encoder_seq_length = max([len(txt) for txt in input_texts])  
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [4]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 15182
Number of unique input tokens: 160
Number of unique output tokens: 126
Max sequence length for inputs: 27
Max sequence length for outputs: 22


In [5]:
input_token_index = dict(
    [(word, i) for i, word in enumerate(input_words)])
target_token_index = dict(
    [(word, i) for i, word in enumerate(target_words)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [6]:
## one hot vector 생성 - [0,1,...0,...0,0]
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    t = MeCab.Tagger('-d /usr/local/lib/mecab/dic/mecab-ko-dic')
  
    parse_intput_sentence = t.parse(input_text).split()
    parse_target_sentence = t.parse(target_text).split()

    for i_word in parse_intput_sentence:
        if i_word.find(',') != -1 or i_word.find('EOS') != -1:
            parse_intput_sentence.remove(i_word)
            
    for t_word in parse_target_sentence:
        if t_word.find(',') != -1 or t_word.find('EOS') != -1:
            parse_target_sentence.remove(t_word)
            
    for t, char in enumerate(parse_intput_sentence[:-1]):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(parse_target_sentence[:-1]):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
        
    

In [13]:
encoder_inputs = Input(shape=(None, num_encoder_tokens)) ##[문장길이, one-hot vector길이]
encoder = LSTM(latent_dim, return_state=True) ## latent_dim == size
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [14]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True) ## return_sequences -> output이 다음 state의 input으로 들어감
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [15]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('model_save/aion_slu_lstm.h5')

Train on 12145 samples, validate on 3037 samples
Epoch 1/100
12145/12145 [==============================] - 7s 616us/step - loss: 0.3463 - val_loss: 0.2327
Epoch 2/100
12145/12145 [==============================] - 6s 522us/step - loss: 0.2026 - val_loss: 0.1617
Epoch 3/100
12145/12145 [==============================] - 6s 521us/step - loss: 0.1203 - val_loss: 0.1035
Epoch 4/100
12145/12145 [==============================] - 6s 522us/step - loss: 0.0747 - val_loss: 0.0726
Epoch 5/100
12145/12145 [==============================] - 6s 525us/step - loss: 0.0423 - val_loss: 0.0267
Epoch 6/100
12145/12145 [==============================] - 6s 523us/step - loss: 0.0224 - val_loss: 0.0039
Epoch 7/100
12145/12145 [==============================] - 6s 525us/step - loss: 0.0119 - val_loss: 0.0011
Epoch 8/100
12145/12145 [==============================] - 6s 523us/step - loss: 0.0085 - val_loss: 3.0202e-04
Epoch 9/100
12145/12145 [==============================] - 6s 525us/step - loss: 0.0074 - v

12145/12145 [==============================] - 6s 523us/step - loss: 2.8866e-08 - val_loss: 2.8847e-08
Epoch 73/100
12145/12145 [==============================] - 6s 526us/step - loss: 2.8694e-08 - val_loss: 2.8678e-08
Epoch 74/100
12145/12145 [==============================] - 6s 524us/step - loss: 2.8523e-08 - val_loss: 2.8506e-08
Epoch 75/100
12145/12145 [==============================] - 6s 525us/step - loss: 2.8376e-08 - val_loss: 2.8408e-08
Epoch 76/100
12145/12145 [==============================] - 6s 523us/step - loss: 2.8266e-08 - val_loss: 2.8290e-08
Epoch 77/100
12145/12145 [==============================] - 6s 525us/step - loss: 2.8159e-08 - val_loss: 2.8183e-08
Epoch 78/100
12145/12145 [==============================] - 6s 524us/step - loss: 2.8046e-08 - val_loss: 2.8070e-08
Epoch 79/100
12145/12145 [==============================] - 6s 530us/step - loss: 2.7916e-08 - val_loss: 2.7931e-08
Epoch 80/100
12145/12145 [==============================] - 6s 523us/step - loss: 2.7

/home/dongjin/anaconda3/envs/my_1.8/lib/python3.5/site-packages/keras/engine/network.py:872: UserWarning: Layer lstm_5 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_4/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_4/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


In [16]:
plot_model(model, to_file='model.png', show_shapes=True)

In [17]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs,
                     [decoder_outputs] + decoder_states)

reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [18]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == "\n" or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [19]:
for seq_index in range(10):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

KeyError: '\t'

In [ ]:
test_input_texts = []

with open("data/aion_test_v.txt", "r", encoding='utf=8') as f:
    reader =f.readlines()
    for i in reader:
        test_input_texts.append(i)

test_encoder_input_data = np.zeros((len(test_input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')

for i, test_input_text in enumerate(test_input_texts):
    for t, char in enumerate(test_input_text):
        test_encoder_input_data[i, t, input_token_index[char]] = 1.
        
for seq_index in range(28, 30):
    input_seq = test_encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', test_input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)